In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import graph_tool.all as gt
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from functions import *


# Graph-Tool compatibility
plt.switch_backend('cairo')
# Style
sns.set_theme(context='talk', style='white', palette='Set2')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# Meta and Parameters

In [3]:
# Load metadata
meta = get_meta()

# Subject preview
filtered = []
for i, row in meta.iterrows():
    try:
        load_graph_by_id(row['SubID'])
        assert not np.isnan(row['nps_MoodDysCurValue'])  # Has NPS information available
        assert row['Sex'] == 'Female'
        assert row['Ethnicity'] != 'White'
        # assert row['BRAAK_AD'] in (3, 4, 5)
    except:
        continue
    filtered.append(f'{row["SubID"]} {row["Ethnicity"]} {row["Sex"]}, {row["Age"]}, BRAAK {row["BRAAK_AD"]}')
filtered = np.sort(filtered)
for i in range(len(filtered)):
    print(filtered[i])

# Parameters
print(f'\nAvailable attention columns: {get_attention_columns()}')
column_ad = get_attention_columns()[0]
column_scz = get_attention_columns()[2]
column_data = get_attention_columns()[4]
synthetic_nodes_of_interest = ['OPC', 'Micro', 'Oligo', 'EN']

M10795 Hispanic Female, 79.0, BRAAK 3.0
M107983 Hispanic Female, 65.0, BRAAK 1.0
M11589 Black Female, 63.0, BRAAK 2.0
M12614 Black Female, 75.0, BRAAK 0.0
M12876 Hispanic Female, 76.0, BRAAK 2.0
M17639 Hispanic Female, 86.0, BRAAK 4.0
M18518 Hispanic Female, 91.0, BRAAK 6.0
M19050 Hispanic Female, 74.0, BRAAK 5.0
M19331 Black Female, 89.0, BRAAK 6.0
M19844 Black Female, 84.0, BRAAK 4.0
M22176 Black Female, 72.0, BRAAK 6.0
M23996 Black Female, 40.0, BRAAK 0.0
M26992 Black Female, 94.0, BRAAK 3.0
M27409 Black Female, 68.0, BRAAK 2.0
M283783 nan Female, 55.0, BRAAK nan
M28379 Hispanic Female, 72.0, BRAAK 6.0
M28437 Hispanic Female, 91.0, BRAAK 6.0
M29263 Black Female, 52.0, BRAAK 0.0
M307567 Black Female, 67.0, BRAAK 1.0
M33390 Hispanic Female, 77.0, BRAAK 2.0
M35312 Black Female, 76.0, BRAAK 0.0
M36388 Black Female, 89.0, BRAAK 3.0
M36634 Hispanic Female, 87.0, BRAAK 6.0
M41136 Black Female, 87.0, BRAAK 6.0
M41496 Black Female, 76.0, BRAAK 4.0
M41581 Black Female, 84.0, BRAAK 3.0
M41973 

# Plots

## Individual Comparisons (Figure 3)

In [4]:
individual_comparisons = [
    # M19050 Hispanic Female, 74.0, BRAAK 5.0
    # M59593 Hispanic Female, 76.0, BRAAK 5.0
    # M72079 Black Female, 64.0, BRAAK 6.0
    # M41496 Black Female, 76.0, BRAAK 4.0
    # M11589 Black Female, 63.0, BRAAK 2.0
    # M73342 Black Female, 62.0, BRAAK 0.0
    # (subject_id_1, subject_id_2, column)
    # for subject_id_1, subject_id_2, column in individual_comparisons:
    ('M19050', 'M59593', column_ad),  # AD - AD
    # ('M72079', 'M41496', column_ad),  # AD - High BRAAK
    # ('M72079', 'M11589', column_ad),  # AD - Low BRAAK
    ('M72079', 'M73342', column_ad),  # AD - CTRL
]
palette = plt.rcParams['axes.prop_cycle'].by_key()['color']
individual_colors = {
    sid: rgba_to_hex(palette[i]) for i, sid in enumerate(
        sum([list(comparison[:2]) for comparison in individual_comparisons], []))
}

# Verify all are available
for subject_id_1, subject_id_2, column in individual_comparisons:
    for sid in [subject_id_1, subject_id_2]:
        load_graph_by_id(sid)

### 3AE Mini Plots (MANUAL)

In [5]:
# SNPs to add (MANUAL)
# TODO: Change to identifying genes, maybe?
graph_snps = {
    'M19050': [('chrx:xxx-xx1:x:x', 'SOX6'), ('chrx:xxx-xx2:x:x', 'TCF12')],
    'M59593': [('chrx:xxx-xx1:x:x', 'SOX6')],
}

for i, (subject_id_1, subject_id_2, column) in enumerate(individual_comparisons):
    print(' - '.join((subject_id_1, subject_id_2, column)))
    
    # Assemble
    sids = [subject_id_1, subject_id_2]
    gs = [compute_graph(load_graph_by_id(sid, column=column)) for sid in sids]

    # Filter
    gs = [
        filter_graph_by_synthetic_vertices(g.copy(), vertex_ids=synthetic_nodes_of_interest)
        for g in gs
    ]
    # TODO: Recalculate once recursive filtering is implemented

    # Add SNPs
    # TODO: Move before filtering once recursive filtering is implemented
    for j, (g, sid) in enumerate(zip(gs, sids)):
        # Skip if no SNPs
        if sid not in graph_snps: continue

        # Add SNPs
        for snp in graph_snps[sid]:
            snp_id, snp_target = snp
            add_snp_to_graph(g, snp_id=snp_id, snp_target=snp_target)
    
    # Plot
    fig, axs = get_mosaic([list(range(2))], scale=9)
    plot_graph_comparison(gs, axs=axs, subject_ids=sids)
    fig.savefig(f'../plots/individual_mini_{"-".join(sids)}_{column}.pdf', format='pdf', transparent=True, backend='cairo')
    print()

M19050 - M59593 - att_D_AD_1
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████████████| 62/62 [00:00<00:00, 205408.25it/s]


Calculating positions...
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████████████| 34/34 [00:00<00:00, 146563.55it/s]


Removing duplicate edges...


...

100%|███████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 202950.19it/s]


Calculating positions...
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 132344.54it/s]


Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 162529.28it/s]

### 3BF Attention Comparisons

In [6]:
for subject_id_1, subject_id_2, column in individual_comparisons:
    print(' - '.join((subject_id_1, subject_id_2, column)))

    # Assemble
    sample_ids = [subject_id_1, subject_id_2]
    graphs = [compute_graph(load_graph_by_id(sid, column=column)) for sid in sample_ids]

    # Get graph
    g = concatenate_graphs(*graphs, threshold=False)
    g = get_intersection(g)
    g = cull_isolated_leaves(g)

    fig, axs = get_mosaic([list(range(1))], scale=6)
    df = plot_individual_edge_comparison(g, sample_ids, ax=axs[0])
    plt.tight_layout()
    fig.savefig(f'../plots/individual_edge_comparison_{"-".join((subject_id_1, subject_id_2))}_{column}.pdf', format='pdf', transparent=True, backend='cairo')
    print()

M19050 - M59593 - att_D_AD_1
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████████| 3327/3327 [00:00<00:00, 250209.78it/s]
/tmp/ipykernel_765/1566030153.py:15: UserWarning: The figure layout has changed to tight
  plt.tight_layout()



M72079 - M73342 - att_D_AD_1
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████████| 3781/3781 [00:00<00:00, 254214.50it/s]


/tmp/ipykernel_765/1566030153.py:15: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


### 3CG Head Comparison

In [7]:
# Parameters
num_edges_per_head = 2

# Compute
all_columns = get_attention_columns()
for subject_id_1, subject_id_2, _ in individual_comparisons:
    print(' - '.join((subject_id_1, subject_id_2, column)))

    # Plot
    fig, axs = get_mosaic([list(range(1))], scale=12)
    plot_head_comparison(
        subject_id_1,
        subject_id_2,
        colors=[individual_colors[sid] for sid in (subject_id_1, subject_id_2)],
        ax=axs[0])
    # plt.tight_layout()
    fig.savefig(f'../plots/individual_head_comparison_{"-".join((subject_id_1, subject_id_2))}_{column}.pdf', format='pdf', transparent=True, backend='cairo')
    print()

M19050 - M59593 - att_D_AD_1
1562 common edges found

M72079 - M73342 - att_D_AD_1
1884 common edges found



### 3DH Pathway Enrichment (MANUAL)

In [11]:
# Enrichment
for subject_id_1, subject_id_2, column in individual_comparisons:
    # Assemble
    sample_ids = [subject_id_1, subject_id_2]
    graphs = [load_graph_by_id(sid, column=column) for sid in sample_ids]
    for i in range(len(graphs)):
        # Remove self-loops
        graphs[i] = graphs[i].loc[graphs[i].apply(lambda x: x['TF'] != x['TG'], axis=1)]

        # Set index, combine tf and tg, rename coef column
        graphs[i].index = graphs[i].apply(lambda x: get_edge_string([x['TF'], x['TG']]), axis=1)
        graphs[i] = graphs[i].drop(columns=['TF', 'TG'])
        graphs[i] = graphs[i].rename(columns={'coef': sample_ids[i]})

        # Drop duplicates
        # TODO: Why are there duplicates?
        graphs[i] = graphs[i].drop_duplicates()

    # Filter to common graphs and join
    graphs = graphs[0].join(graphs[1], how='inner')

    # Get differentially expressed genes
    graphs['Difference'] = np.abs(graphs[subject_id_1] - graphs[subject_id_2])
    graphs = graphs.sort_values('Difference', ascending=False)

    # Write to file
    fname_prefix = f'../plots/individual_pathway_enrichment_{"-".join((subject_id_1, subject_id_2))}_{column}'
    f_tfs = open(fname_prefix + '.tfs.txt', 'w')
    f_tgs = open(fname_prefix + '.tgs.txt', 'w')
    f_edges = open(fname_prefix + '.edges.txt', 'w')
    for edge in graphs.index:
        # TODO: Only show unique tf, tg, maybe?
        f_edges.write(edge + '\t' + f'{graphs.loc[edge, "Difference"]:.5f}' + '\n')
        tf, tg = edge.split(get_edge_string())
        # Don't show synthetic nodes
        if not string_is_synthetic(tf): f_tfs.write(tf + '\n')
        if not string_is_synthetic(tg): f_tgs.write(tg + '\n')
    f_tfs.close()
    f_tgs.close()
    f_edges.close()

    # MANUAL PROCESSING
    # Run the output from '<fname_prefix>.xxs.txt' in DisGeNet, save file
    # from Enrichment_QC/GO_DisGeNET as '<fname_prefix>.xxs.csv'

    # Plot enrichments
    get_enrichment_file = lambda x: f'{fname_prefix}.{x}'
    for ftype in ['tfs', 'tgs']:
        # Get file name
        fname = get_enrichment_file(ftype)

        # Get enrichment
        enrichment = get_enrichment(fname)
        if enrichment is None: continue

        # Plot
        fig, axs = get_mosaic([list(range(1))], scale=9)
        plot_enrichment(enrichment, ax=axs[0])
        fig.savefig(fname + '.pdf', format='pdf', transparent=True, backend='cairo')

        # Plot
        fig, axs = get_mosaic([list(range(1))], scale=9)
        plot_enrichment(enrichment, ax=axs[0])
        fig.savefig(fname + '.pdf', format='pdf', transparent=True, backend='cairo')

### 3IJ Population Variance Analysis

In [9]:
# Get variance for each head over all edges
all_columns = get_attention_columns()
subject_ids = list(meta['SubID'])
value_name = 'Variance'

df_all = pd.DataFrame(columns=['Edge', 'Head', value_name])
for column in all_columns:
    print(column)

    # Compute variance
    df_subgroup = compute_contrast_summary(
        {'Population': subject_ids},
        column=column,
        population=False)

    # Format df
    df = df_subgroup['Population'][['Edge', value_name]].copy()
    df['Head'] = column
    df_all = pd.concat([df_all, df])

    print()

# Format
df_all_pivot = df_all.pivot(index='Edge', columns='Head', values=value_name)

# Get top edges
idx_to_include = get_top_idx(df_all_pivot.abs(), all_columns, num_edges_per_head=2)

# Plot curves and circle heatmap
fig, axs = get_mosaic([list(range(1))], scale=9)
plot_contrast_curve(df_all, subgroup_name='Head', value_name=value_name, sorting_subgroup='Mean', concatenate=False, ax=axs[0])
axs[0].set_ylabel('Attention Variance')
fig.savefig(f'../plots/individual_population_curves.pdf', format='pdf', transparent=True, backend='cairo')
fig, axs = get_mosaic([list(range(1))], scale=9)
plot_circle_heatmap(df_all_pivot.iloc[idx_to_include], column_name='Head', index_name='Edge', value_name=value_name, ax=axs[0])
axs[0].set_xlabel(None); axs[0].set_ylabel(None)
plt.tight_layout()
fig.savefig(f'../plots/individual_population_heatmap.pdf', format='pdf', transparent=True, backend='cairo')

att_D_AD_1
No threshold provided, using threshold of 0.013423301698535669.
Removing duplicate edges...


100%|█████████████████████████████████████████████████████████████████████| 1043828/1043828 [00:03<00:00, 342779.36it/s]
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Filtered from 5047 vertices and 374869 edges to 1664 vertices and 13275 edges via common edge filtering.


100%|███████████████████████████████████████████████████████████████████████████| 13275/13275 [00:02<00:00, 5879.28it/s]



att_D_AD_2
No threshold provided, using threshold of 0.013423301698535669.
Removing duplicate edges...


...


att_D_no_prior_1
No threshold provided, using threshold of 0.013423301698535669.
Removing duplicate edges...


100%|█████████████████████████████████████████████████████████████████████| 1043828/1043828 [00:03<00:00, 343529.74it/s]
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Filtered from 5047 vertices and 374869 edges to 1664 vertices and 13269 edges via common edge filtering.


100%|███████████████████████████████████████████████████████████████████████████| 13269/13269 [00:02<00:00, 5867.16it/s]


/tmp/ipykernel_765/4202816374.py:37: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


### 3K Dosage Analysis (MANUAL)

In [8]:
# Get dosage information
dosage = get_dosage()

# Parameters
column = column_data
subject_ids = meta['SubID'].to_numpy()

# Get most variant edges
graphs = []; graph_sids = []
for sid in subject_ids:
    try:
        graphs.append(compute_graph(load_graph_by_id(sid, column=column)))
        graph_sids.append(sid)
    except: pass
variance = compute_edge_summary(graphs=graphs, subject_ids=graph_sids)[0]
variance = variance.set_index('Edge')
variance = variance.std(axis=1)
target_edges_all = variance.index[np.argsort(variance)[::-1]]
target_edges_all = np.array(target_edges_all)

# CLI
print()
for i, edge in enumerate(target_edges_all[:20]):
    print(f'{i}. {edge}')

/mnt/c/Users/nck/repos/GNN-Plus/Attention Analysis/scripts/functions/file.py:109: DtypeWarning: Columns (20,24,25,57,59,61,90,145,231,249,270,271,286,292,297,318,355,397,437,510,511,627,632,646,662,702,732,738,761,786,804,805,833,837,868,891,913,918,924,953,954,955,956,957,958) have mixed types. Specify dtype option on import or set low_memory=False.
  dosage = pd.read_csv(DOSAGE)


No threshold provided, using threshold of 0.013423301698535669.
Removing duplicate edges...


100%|█████████████████████████████████████████████████████████████████████| 1043828/1043828 [00:02<00:00, 349175.90it/s]
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Filtered from 5047 vertices and 374869 edges to 1664 vertices and 13268 edges via common edge filtering.


100%|███████████████████████████████████████████████████████████████████████████| 13268/13268 [00:02<00:00, 6002.36it/s]



0. Micro -- OPC
1. Micro -- Astro
2. Micro -- IN_LAMP5_RELN
3. Micro -- Oligo
4. Micro -- EN_L2_3_IT
5. Micro -- IN_LAMP5_LHX6
6. Micro -- EN_L3_5_IT_1
7. Micro -- EN_L6_IT_1
8. Micro -- EN_L6_CT
9. Micro -- EN_L3_5_IT_2


In [10]:
# COMPUTED MANUALLY USING `target_genes`
target_edges = target_edges_all[[12, 13, 16]]
chromosomes = ['10', '1', '1']

# Plot
# TODO: Find out why there are gaps in the plot,
# TODO: Right now FDR finds no significant SNPs, find another adjustment
#   likely to do with `get_chromosome_lengths`
for i, (target_edge, chromosome) in enumerate(zip(target_edges, chromosomes)):
    print(target_edge)
    fig, axs = get_mosaic([3*[0]], scale=9)
    df = plot_attention_dosage_correlation(
        dosage,
        # n=100,
        meta=meta,
        subject_ids=subject_ids,
        column=column,
        target_edge=target_edge,
        chromosomes=[chromosome] if chromosome is not None else None,
        ax=axs[0])
    plt.tight_layout()
    fig.savefig(f'../plots/individual_dosage_correlation_{target_edge}.pdf', format='pdf', transparent=True, backend='cairo')
    print()

SOX6 -- PCDH15
Converting dosage ids to subject ids...


/mnt/c/Users/nck/repos/GNN-Plus/Attention Analysis/scripts/functions/computation.py:398: UserWarning: Unable to find SNP 'BM-44-4996' in metadata
  warnings.warn(f'Unable to find SNP \'{dosage_id}\' in metadata')
/mnt/c/Users/nck/repos/GNN-Plus/Attention Analysis/scripts/functions/computation.py:398: UserWarning: Unable to find SNP 'RUSH-043
' in metadata
  warnings.warn(f'Unable to find SNP \'{dosage_id}\' in metadata')


Computing correlations...


100%|████████████████████████████████████████████████████████████████████████| 502282/502282 [00:23<00:00, 21697.66it/s]
/tmp/ipykernel_827/2425618143.py:21: UserWarning: The figure layout has changed to tight
  plt.tight_layout()



Immune -- RUNX3
Converting dosage ids to subject ids...


...

100%|████████████████████████████████████████████████████████████████████████| 502282/502282 [00:41<00:00, 12236.02it/s]
/tmp/ipykernel_827/2425618143.py:21: UserWarning: The figure layout has changed to tight
  plt.tight_layout()



Micro -- TAL1
Converting dosage ids to subject ids...


/mnt/c/Users/nck/repos/GNN-Plus/Attention Analysis/scripts/functions/computation.py:398: UserWarning: Unable to find SNP 'BM-44-4996' in metadata
  warnings.warn(f'Unable to find SNP \'{dosage_id}\' in metadata')
/mnt/c/Users/nck/repos/GNN-Plus/Attention Analysis/scripts/functions/computation.py:398: UserWarning: Unable to find SNP 'RUSH-043
' in metadata
  warnings.warn(f'Unable to find SNP \'{dosage_id}\' in metadata')


Computing correlations...


100%|████████████████████████████████████████████████████████████████████████| 502282/502282 [00:39<00:00, 12772.98it/s]
/tmp/ipykernel_827/2425618143.py:21: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


## Group Comparisons (Figure 4)

In [12]:
# Combinations
# TODO: Potentially move each entry to dictionary, so changes in order
#   are easier to propagate
contrast_groupings = [
    # (contrast name, contrast group, attention column, comparison column, target meta column)
    # for contrast_name, contrast_group, column, comparison, target in contrast_groupings:
    # TODO: Revise ethnicity prediction
    ('c06x', 'AD', column_data, column_ad, 'Ethnicity'),  # Maybe make this all subjects?  Or just control?
    ('c06x', 'AD', column_ad, column_data, 'BRAAK_AD'),  # Eventually SCZ, BP and such
    ('c71x', 'MoodDys', column_data, column_ad, 'nps_MoodDysCurValue'),  # Dysphoria
    ('c72x', 'DecInt', column_data, column_ad, 'nps_DecIntCurValue'),  # Anhedonia
]

### 4A BRAAK Comparison

In [13]:
# TODO: Use highest variance rather than random edges
# TODO: Make both columns same scale, maybe?
for contrast_name, _, column, comparison, target in contrast_groupings:
    print(' - '.join((contrast_name, column, target)))
    # Get contrast
    contrast = get_contrast(contrast_name)

    # Plot
    fig, axs = get_mosaic([6*[0], 6*[1]], scale=3)
    sns.despine()

    _, edges_include = plot_BRAAK_comparison(
        contrast,
        # {k: v[:10] for k, v in contrast.items()},
        meta=meta,
        column=column,
        target=target,
        legend=False,
        ax=axs[0])
    plt.xlabel(None)
    plt.ylabel(column)
    plt.xticks([])
    
    plot_BRAAK_comparison(
        contrast,
        meta=meta,
        column=comparison,
        target=target,
        edges_include=edges_include,
        ax=axs[1])
    plt.ylabel(comparison)

    fig.savefig(f'../plots/group_differential_expression_{contrast_name}_{column}_{comparison}_{target}.pdf', format='pdf', transparent=True, backend='cairo')
    print()

c06x - att_D_no_prior_0 - Ethnicity
No threshold provided, using threshold of 0.021198411899135766.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 522653/522653 [00:01<00:00, 340015.97it/s]
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Filtered from 4997 vertices and 221958 edges to 1319 vertices and 8839 edges via common edge filtering.


100%|████████████████████████████████████████████████████████████████████████████| 8839/8839 [00:00<00:00, 10518.20it/s]


No threshold provided, using threshold of 0.021198411899135766.
Removing duplicate edges...


...

100%|████████████████████████████████████████████████████████████████████████████| 5652/5652 [00:00<00:00, 15897.87it/s]


No threshold provided, using threshold of 0.031252225557032644.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 297554/297554 [00:00<00:00, 347348.43it/s]
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Filtered from 4942 vertices and 144619 edges to 993 vertices and 5655 edges via common edge filtering.


100%|████████████████████████████████████████████████████████████████████████████| 5655/5655 [00:00<00:00, 17229.03it/s]


### 4B Cross-Validation Accuracies

In [14]:
# TODO: Make all y-labels horizontal
for contrast_name, _, column, _, target in contrast_groupings:
    # if contrast_name != 'c71x': continue
    print(' - '.join((contrast_name, column, target)))
    # Get contrast
    contrast = get_contrast(contrast_name)

    # Compute prioritized edges
    # Get 100 most variant edges
    # TODO: Revise this method, maybe also consider means
    sids = sum([sids for _, sids in contrast.items()], [])
    df_subgroup = compute_contrast_summary(contrast, column=column)
    df = join_df_subgroup(df_subgroup, num_sort=100)
    prioritized_edges = list(df.index)

    # Plot
    # TODO: Maybe return to row-normalization
    fig, axs = get_mosaic([2*[0]], scale=9)
    df, acc = plot_prediction_confusion(contrast, meta=meta, column=column, target=target, prioritized_edges=prioritized_edges, classifier_type='SGD', ax=axs[0])
    
    # Save plot
    fname_prefix = f'../plots/group_prioritized_edge_prediction_{contrast_name}_{column}_{target}'
    fig.savefig(f'{fname_prefix}.pdf', format='pdf', transparent=True, backend='cairo')

    # Save text
    f_edges = open(f'{fname_prefix}.edges.txt', 'w')
    f_tfs = open(f'{fname_prefix}.tfs.txt', 'w')
    f_tgs = open(f'{fname_prefix}.tgs.txt', 'w')
    for edge in prioritized_edges:
        f_edges.write(edge + '\n')
        tf, tg = edge.split(get_edge_string(['', '']))
        f_tfs.write(tf + '\n')
        f_tgs.write(tg + '\n')
    f_edges.close()
    f_tfs.close()
    f_tgs.close()

    # CLI
    print()

c06x - att_D_no_prior_0 - Ethnicity
No threshold provided, using threshold of 0.02839460354258344.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 335051/335051 [00:00<00:00, 343979.63it/s]
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Filtered from 4943 vertices and 154650 edges to 1134 vertices and 6814 edges via common edge filtering.
No threshold provided, using threshold of 0.04228647070516764.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 187602/187602 [00:00<00:00, 343998.83it/s]
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Filtered from 4830 vertices and 99726 edges to 829 vertices and 4368 edges via common edge filtering.


...

100%|████████████████████████████████████████████████████████████████████████████| 5652/5652 [00:00<00:00, 16950.58it/s]


No threshold provided, using threshold of 0.031252225557032644.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 297554/297554 [00:00<00:00, 350479.11it/s]
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Filtered from 4942 vertices and 144619 edges to 993 vertices and 5652 edges via common edge filtering.


100%|████████████████████████████████████████████████████████████████████████████| 5652/5652 [00:00<00:00, 17488.13it/s]


### 4CD Characteristic Curves

In [15]:
# TODO: Separate into two plots
# Get plots for each column
for contrast_name, _, column, _, _ in contrast_groupings:
    print(' - '.join((contrast_name, column)))

    # Get contrast
    contrast = get_contrast(contrast_name)

    # Compute
    df_subgroup = compute_contrast_summary(contrast, column=column)

    # Plot heatmap, individually-sorted, and pop-sorted
    # plot_subgroup_heatmap(df_subgroup, ax=axs[0])
    # axs[1].get_shared_x_axes().join(axs[1], axs[2])
    # plot_contrast_curve(df_subgroup, ax=axs[1], legend=False)  # Individually sorted
    # plot_contrast_curve(df_subgroup, sorting_subgroup='Population', ax=axs[1])  # Population sorted
    # axs[2].set_ylabel(None)

    # Plot mean-sorted
    fig, axs = get_mosaic([list(range(1))], scale=9)
    plot_subgroup_heatmap(df_subgroup, ax=axs[0])
    plt.tight_layout()
    fig.savefig(f'../plots/group_variance_heatmap_{contrast_name}_{column}.pdf', format='pdf', transparent=True, backend='cairo')
    fig, axs = get_mosaic([list(range(1))], scale=9)
    plot_contrast_curve(df_subgroup, sorting_subgroup='Mean', ax=axs[0])  # Mean sorted
    plt.tight_layout()
    fig.savefig(f'../plots/group_variance_curves_{contrast_name}_{column}.pdf', format='pdf', transparent=True, backend='cairo')
    print()

c06x - att_D_no_prior_0
No threshold provided, using threshold of 0.02839460354258344.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 335051/335051 [00:00<00:00, 349004.34it/s]
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Filtered from 4943 vertices and 154650 edges to 1134 vertices and 6814 edges via common edge filtering.
No threshold provided, using threshold of 0.04228647070516764.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 187602/187602 [00:00<00:00, 348181.46it/s]
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Filtered from 4830 vertices and 99726 edges to 829 vertices and 4368 edges via common edge filtering.


...

100%|████████████████████████████████████████████████████████████████████████████| 3159/3159 [00:00<00:00, 31210.46it/s]


No threshold provided, using threshold of 0.031252225557032644.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 297554/297554 [00:00<00:00, 315417.65it/s]
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Filtered from 4942 vertices and 144619 edges to 993 vertices and 5652 edges via common edge filtering.


100%|████████████████████████████████████████████████████████████████████████████| 5652/5652 [00:00<00:00, 16963.56it/s]
/tmp/ipykernel_765/1001339706.py:22: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/tmp/ipykernel_765/1001339706.py:26: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


### 4E Group Enrichment (MANUAL)

In [12]:
# MANUAL PROCESSING
# Run the output from 4B '<fname_prefix>.xxs.txt' in DisGeNet, save file
# from Enrichment_QC/GO_DisGeNET as '<fname_prefix>.xxs.csv'

for contrast_name, _, column, _, target in contrast_groupings:
    # Make fname
    fname_prefix = f'../plots/group_prioritized_edge_prediction_{contrast_name}_{column}_{target}'

    # Plot enrichments
    get_enrichment_file = lambda x: f'{fname_prefix}.{x}'
    for ftype in ['tfs', 'tgs']:
        # Get file name
        fname = get_enrichment_file(ftype)

        # Get enrichment
        enrichment = get_enrichment(fname)
        if enrichment is None: continue

        # Plot
        fig, axs = get_mosaic([list(range(1))], scale=9)
        plot_enrichment(enrichment, ax=axs[0])
        fig.savefig(fname + '.pdf', format='pdf', transparent=True, backend='cairo')

NameError: name 'contrast_groupings' is not defined

## General

### Enrichment

In [17]:
# Generate fake enrichment data
import itertools
# Get columns
cell_type = synthetic_nodes_of_interest
disease = ['Alzheimer\'s', 'Schizophrenia', 'Bipolar', 'Depression', 'Weight Loss', 'Sleeplessness']
combined = [val for val in itertools.product(cell_type, disease)]
cell_type = [val[0] for val in combined]
disease = [val[1] for val in combined]
# Get significance
np.random.seed(42)
significance = np.exp(-8 * np.random.rand(len(combined)))
# Combine
df = pd.DataFrame({'cell_type': cell_type, 'disease': disease, 'significance': significance})
df = df.loc[df['significance'] < 5e-2]

# Rename
df = df.rename(columns={'cell_type': 'Cell Type', 'disease': 'Disease'})
# Add significance scale
df['-log(p)'] = -np.log10(df['significance'])
# Plot
fig, axs = get_mosaic([list(range(1))], scale=9)
plot_enrichment(df, ax=axs[0])
plt.savefig(f'../plots/enrichment.pdf', format='pdf', transparent=True, backend='cairo')

### Graph Legend

In [18]:
# Plot legend
plt.clf()
plot_legend()
plt.gca().axis('off')
plt.tight_layout()
plt.savefig(f'../plots/graph_legend.pdf', format='pdf', transparent=True, backend='cairo')

/tmp/ipykernel_765/2458235525.py:5: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


# Archive

## Individual Comparisons

### 3I SNP Trend Analysis

In [19]:
# Get top variant edges across population, and values for all heads
# TODO

# For all edges
# TODO
    # Get 100 closest SNPs for TG
    # TODO

    # For all close SNPs
    # TODO
        # For all heads
        # TODO
            # Assess mean separation across SNP variants
            # TODO

            # Add mean separation, edge, SNP, head to LIST
            # TODO
    
# Choose best mean separation results from LIST
# TODO

# Plot SNP variants vs attention
# TODO

In [20]:
# # Fake data placeholder
# for i in range(3):
#     np.random.seed(42+i)

#     # Fake dataframe for a single SNP
#     n = 200
#     variants = ['AATG', 'ACTG', 'GATG', 'GGTG', 'AATA', 'AAGG']
#     ch = np.random.randint(23) + 1
#     loc = np.random.randint(10000, 1000000)
#     loc = f'{ch}:{loc}-{loc+len(variants[0])}'
#     df = pd.DataFrame({
#         'Subject ID': np.random.choice(range(int(n/10)), n, replace=True),
#         'SNP Variant': np.random.choice(variants, n, replace=True),
#         'Head': np.random.choice(get_attention_columns(), n, replace=True),
#         'Attention Weight': np.random.rand(n) / 10,
#     })
#     df = df.sort_values(['Head'])  # Enforce x ordering

#     # Plot
#     fig, axs = get_mosaic([2*[0]], scale=9)

#     sns.boxplot(data=df, x='Head', y='Attention Weight', hue='SNP Variant', ax=axs[0])
#     axs[0].set_title(f'SNP: {loc}'); axs[0].set_xlabel(None)
#     plt.xticks(rotation=90)
#     plt.legend(bbox_to_anchor=(1.02, .7), loc='upper left', borderaxespad=0, frameon=False)

#     plt.tight_layout()
#     fig.savefig(f'../plots/individual_SNP_analysis_{i}.pdf', format='pdf', transparent=True, backend='cairo')

### Coex Individual Trio Comparison

In [21]:
# # Choose three graphs
# graphs = coex_g_individuals[:3]
# graphs_subject_ids = individual_subject_ids[:3]

# # Create figure
# fig, axs = get_mosaic([list(range(len(graphs)+1))], scale=9)

# # Compute edge summaries
# df, concatenated_graph = compute_edge_summary(graphs=graphs, subject_ids=graphs_subject_ids)

# # Show individual graph comparisons
# plot_graph_comparison(graphs, axs=axs, subject_ids=graphs_subject_ids)

# # Show edge summary
# plot_edge_summary(graphs, df=df, ax=axs[len(graphs)], subject_ids=graphs_subject_ids)

# # Save figure
# plt.tight_layout()
# fig.savefig(f'../plots/CoexIndividualTrioComparison.pdf', format='pdf', transparent=True, backend='cairo')

### Individual Trio Comparison

In [22]:
# # Choose three graphs
# graphs = data_g_individuals[:3]
# graphs_subject_ids = individual_subject_ids[:3]

# # Create figure
# plt.clf()
# fig, axs = get_mosaic([list(range(len(graphs)+1))], scale=9)

# # Compute edge summaries
# df, concatenated_graph = compute_edge_summary(graphs=graphs, subject_ids=graphs_subject_ids)

# # Show individual graph comparisons
# plot_graph_comparison(graphs, axs=axs, subject_ids=graphs_subject_ids)

# # Show edge summary
# plot_edge_summary(graphs, df=df, ax=axs[len(graphs)], subject_ids=graphs_subject_ids)

# # Save figure
# plt.tight_layout()
# fig.savefig(f'../plots/IndividualTrioComparison.pdf', format='pdf', transparent=True, backend='cairo')

## Group Comparisons

### 4A Group Visualizations

In [23]:
# for contrast_name, contrast_group, column, _ in contrast_groupings:
#     print(' - '.join((contrast_name, contrast_group, column)))
#     # Get contrast
#     contrast = get_contrast(contrast_name)

#     # Calculate aggregate graph
#     all_graphs, _ = load_many_graphs(contrast[contrast_group], column=column_ad)
#     all_graphs = [compute_graph(graph) for graph in all_graphs]
#     aggregate_graph = concatenate_graphs(
#         *all_graphs,
#         threshold=True,
#         color_by_source=False,
#         # remove_duplicate_edge=False,
#     )

#     # Filter graph
#     aggregate_graph = filter_graph_by_synthetic_vertices(aggregate_graph, vertex_ids=synthetic_nodes_of_interest)

#     # Plot
#     fig, axs = get_mosaic([list(range(1))], scale=9)

#     ax = axs[0]
#     visualize_graph_base(aggregate_graph, pos=get_graph_pos(aggregate_graph), mplfig=ax)
#     ax.axis('off')

#     fig.savefig(f'../plots/group_aggregate_{contrast_name}_{contrast_group}_{column}.pdf', format='pdf', transparent=True, backend='cairo')
#     print()

### Aggregate Trio Comparison

In [24]:
# # Parameters
# contrast = 'c01x'
# column = column_ad

# # Create figure
# fig, axs = get_mosaic([list(range(len(get_contrast(contrast))+1))], scale=9)

# # Compute aggregate edge summaries
# contrast_group = compute_aggregate_edge_summary(get_contrast(contrast), column=column_ad)

# # Plot graph comparison
# plot_graph_comparison(
#     graphs=[v for k, v in contrast_group[0].items()],
#     subject_ids=[k for k, v in contrast_group[1].items()],
#     axs=[axs[i] for i in range(len(get_contrast(contrast)))])

# # Plot edge summary for subgroups
# plot_aggregate_edge_summary(ax=axs[len(get_contrast(contrast))], contrast=contrast_group)

# # Save figure
# plt.tight_layout()
# fig.savefig(f'../plots/AggregateTrioComparison.pdf', format='pdf', transparent=True, backend='cairo')

##### Linkage Analysis

In [25]:
# # Record edge instances
# # df = pd.DataFrame(columns=['Edge', 'Subgroup', 'Count'])
# df = {k: [] for k in ['Edge', 'Subgroup', 'Count']}
# for subgroup in contrast_group[0]:
#     g = contrast_group[0][subgroup]
#     for e in tqdm(g.edges(), total=g.num_edges()):
#         coefs = g.ep.coefs[e]
#         row = [get_edge_string(g, e), subgroup, sum([c!=0 for c in coefs])]
#         # df.loc[df.shape[0]] = row  # Slow
#         for k, v in zip(df, row):
#             df[k].append(v)
# df = pd.DataFrame(df)

# # Get edge counts
# count_table = df.pivot(index='Edge', columns='Subgroup', values='Count')
# count_table = count_table.fillna(0)
# # Max scale for fairness
# for subgroup in contrast_group[0]:
#     count_table[subgroup] /= count_table[subgroup].max()
# # Compute differences
# # TODO: REVISE DIFFERENCE METRIC
# count_table['Difference'] = count_table['AD'] - count_table['Control']
# count_table['Range'] = count_table.max(axis=1) - count_table.min(axis=1)

# # Get list of linkages by significance
# open(f'../plots/AggregateTrioComparisonList.txt', 'w').close()
# for i in np.unique(count_table['Difference'])[::-1]:
#     condition = (count_table['Difference'] == i)
#     significant_edges = list(count_table.loc[condition].index)
#     synthetic_genes = np.concatenate([detect_synthetic_vertices_graph(contrast_group[0][subgroup]) for subgroup in contrast_group[0]])
#     try: significant_genes = np.concatenate([e.split('--') for e in significant_edges])
#     except: significant_genes = []
#     significant_genes = np.unique([g for g in significant_genes if g not in synthetic_genes])

#     # Print significant genes
#     if len(significant_genes) > 0:
#         with open(f'../plots/AggregateTrioComparisonList.txt', 'a') as f:
#             print(f'--- {i} ---', file=f)
#             for g in significant_genes:
#                 print(g, file=f)
#             print(file=f)

### Differentially Expressed Edges

In [26]:
# # TODO: Fix nodes cutting off
# # Plot total and subplots for aggregate differences
# for prefix, individuals in zip(('diff', 'data'), (diff_g_individuals, data_g_individuals)):
#     plt.clf()
#     concat = concatenate_graphs(*individuals)
#     concat = get_intersection(concat)
#     concat = cull_isolated_leaves(concat)
#     concat = remove_text_by_centrality(concat)
#     concat = color_by_significance(concat)
#     visualize_graph(concat)
#     plt.gca().axis('off')
#     plt.tight_layout()
#     plt.savefig(f'../plots/{prefix}_concat.pdf', format='pdf', transparent=True, backend='cairo')

#     # Show all subsets of graph by cell type
#     for v_name in detect_synthetic_vertices_graph(concat):
#         plt.clf()
#         subset = subset_by_hub(concat, [v_name])
#         visualize_graph(subset)
#         plt.gca().axis('off')
#         plt.tight_layout()
#         plt.savefig(f'../plots/{prefix}_concat_{v_name}.pdf', format='pdf', transparent=True, backend='cairo')